In [1]:
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from pyiomica_master import pyiomica as pio
from pyiomica_master.pyiomica import visualizationFunctions
from pyiomica_master.pyiomica import visibilityGraphCommunityDetection
import matplotlib as mpl
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tqdm import tqdm
%matplotlib inline

Loading PyIOmica 1.3.3 (https://github.com/gmiaslab/pyiomica by G. Mias Lab)


In [2]:
x_frame1_05 = np.genfromtxt('/home/nvekariy/signal_code/datasets/neural_4_class/x_frame1_1.25_seed.csv', delimiter=',')
x_frame2_05 = np.genfromtxt('/home/nvekariy/signal_code/datasets/neural_4_class/x_frame2_1.25_seed.csv', delimiter=',')
x_frame3_05 = np.genfromtxt('/home/nvekariy/signal_code/datasets/neural_4_class/x_frame3_1.25_seed.csv', delimiter=',')
x_framen_05= np.genfromtxt('/home/nvekariy/signal_code/datasets/neural_4_class/x_frame_noise_1.25_seed.csv', delimiter=',')

In [3]:
x=np.concatenate((x_frame1_05,x_frame2_05,x_frame3_05,x_framen_05[:6000]),axis=0)
y=np.concatenate((np.zeros(len(x_frame1_05)),np.zeros(len(x_frame2_05))+1,np.zeros(len(x_frame3_05))+2,np.zeros(len(x_framen_05[:6000]))+3),axis=0)

In [4]:
X, Y = shuffle(x,y, random_state=1)

In [5]:
def calculate_adj(data):
    temp1=[]
    for p, i in enumerate(data):
        #print(f'Converting ==> {p+1} \r', end="",flush=True)
        vs_out = visibilityGraphCommunityDetection.createVisibilityGraph(i, range(len(i)), "dual_natural", weight='tan')
        vs_out=np.asarray(vs_out)
        temp1.append(vs_out)
        
    return temp1

In [6]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)

Number of CPU cores: 48


In [7]:
df=np.array_split(X, num_cores)
print(len(df[0]))

533


In [8]:
with Pool(processes=num_cores) as pool:
    results = pool.map(calculate_adj, df)

pool.close()
pool.join()

In [9]:
x_adj=[]
for submatrix in results:
    x_adj.extend(submatrix)

In [10]:
print(len(x_adj))

25578


In [11]:
x_adj=np.array(x_adj)

In [12]:
def graph_index_complexity(matrix):
    N = len(matrix) # Number of nodes
    E = np.sum(matrix > 0) / 2 # Number of edges (since matrix is symmetric for undirected graphs)
    
    if N == 0 or E == 0:
        return 0
    
    GIC = (E / N) * np.log(N)
    return GIC

In [13]:
import networkx as nx
def runall(data):
    temp1=[]
    for i in data:
        G = nx.from_numpy_array(i, parallel_edges=False)
        gic=[graph_index_complexity(i)]
        cpl=[nx.average_shortest_path_length(G)]
        ge=[nx.global_efficiency(G)]
        le=[nx.local_efficiency(G)]
        ac=[nx.degree_assortativity_coefficient(G)]
        cc=list(nx.clustering(G).values())
        temp1.append(gic+cpl+ge+le+ac+cc) #+cpl+ge+le+ac+cc
    return temp1

In [14]:
from multiprocessing import Pool
import multiprocessing

if __name__ == '__main__':
    num_cores = multiprocessing.cpu_count()
    print("Number of CPU cores:", num_cores)


df1=np.array_split(x_adj[:], num_cores)


print(len(df[0]))




Number of CPU cores: 48
533


In [15]:

import time
start_time = time.time()
with Pool(processes=num_cores) as pool1:
    results1 = pool1.map(runall, df1)

pool1.close()
pool1.join()

print("--- %s seconds ---" % (time.time() - start_time))


--- 28.9857017993927 seconds ---


In [16]:
x_g=[]
for submatrix in results1:
    x_g.extend(submatrix)

In [17]:
x_g=np.array(x_g)


In [18]:
print(x_g.shape)

(25578, 61)


In [19]:
x_train, x_temp, y_train, y_temp = train_test_split(x_g, Y, test_size=0.30, random_state=42)
x_val,x_test,y_val,y_test=train_test_split(x_temp,y_temp,test_size=0.50, random_state=42)

In [20]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(17904, 61)
(3837, 61)
(3837, 61)


In [21]:
import tensorflow as tf


2023-08-13 10:09:40.252460: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-13 10:09:42.258885: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-13 10:09:43.398777: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-

In [22]:
input_shape = (x_train.shape[1],1)
num_classes = 4
model3 = tf.keras.models.Sequential()
model3.add(tf.keras.layers.Conv1D(filters= 32, kernel_size=3, activation='relu',padding='same',input_shape= input_shape))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))

# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
# model3.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
# model3.add(tf.keras.layers.MaxPooling1D(pool_size=2,strides=None,padding='valid'))
model3.add(tf.keras.layers.Flatten())

model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dropout(0.1))
model3.add(tf.keras.layers.Dense(100, activation='relu'))
model3.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

2023-08-13 10:10:10.710432: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-08-13 10:10:10.710512: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-13 10:10:10.710542: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c0188): /proc/driver/nvidia/version does not exist
2023-08-13 10:10:10.711555: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow

In [23]:
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 61, 32)            128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 30, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 30, 32)            3104      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 15, 32)           0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 480)               0         
                                                                 
 dense (Dense)               (None, 100)               4

In [24]:
model3.compile(loss='SparseCategoricalCrossentropy', optimizer='Adam', metrics=['acc'])


In [25]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model



checkpoint = ModelCheckpoint('F1.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


In [26]:
history_log = model3.fit(x_train, y_train, epochs=30, verbose=1, validation_data=(x_test, y_test), callbacks=[checkpoint])


Epoch 1/30
560/560 [==============================] - 4s 4ms/step - loss: 0.4869 - acc: 0.7985 - val_loss: 0.2478 - val_acc: 0.8989
Epoch 2/30
560/560 [==============================] - 2s 3ms/step - loss: 0.2594 - acc: 0.8947 - val_loss: 0.2144 - val_acc: 0.9153
Epoch 3/30
560/560 [==============================] - 2s 3ms/step - loss: 0.2297 - acc: 0.9101 - val_loss: 0.1960 - val_acc: 0.9242
Epoch 4/30
560/560 [==============================] - 2s 3ms/step - loss: 0.2145 - acc: 0.9174 - val_loss: 0.1916 - val_acc: 0.9252
Epoch 5/30
560/560 [==============================] - 2s 3ms/step - loss: 0.2105 - acc: 0.9179 - val_loss: 0.1930 - val_acc: 0.9239
Epoch 6/30
560/560 [==============================] - 2s 3ms/step - loss: 0.2040 - acc: 0.9215 - val_loss: 0.1855 - val_acc: 0.9268
Epoch 7/30
560/560 [==============================] - 2s 3ms/step - loss: 0.1969 - acc: 0.9231 - val_loss: 0.1862 - val_acc: 0.9249
Epoch 8/30
560/560 [==============================] - 2s 3ms/step - loss: 0.

Epoch 29/30
560/560 [==============================] - 2s 3ms/step - loss: 0.0977 - acc: 0.9626 - val_loss: 0.2536 - val_acc: 0.9213
Epoch 30/30
560/560 [==============================] - 2s 3ms/step - loss: 0.0924 - acc: 0.9654 - val_loss: 0.2601 - val_acc: 0.9192


In [27]:
model3.save_weights('/home/nvekariy/signal_code/New_neural_code_SNR_1.25_h/F1.h5')


In [28]:
model3.load_weights('/home/nvekariy/signal_code/New_neural_code_SNR_1.25_h/F1.h5')


In [29]:
predictions = model3.predict(x_test)
pred_arg = np.argmax(predictions, axis=1)

120/120 [==============================] - 0s 1ms/step


In [30]:
from sklearn.metrics import confusion_matrix
cnf_matrix=confusion_matrix(y_test,pred_arg)
print("cnf_matrix_2.0_70")
print("")
print(cnf_matrix)

cnf_matrix_2.0_70

[[930   1   1  31]
 [  0 863 108  30]
 [  0  94 888   0]
 [ 33  11   1 846]]


In [31]:
from sklearn.metrics import accuracy_score, recall_score,precision_score,roc_auc_score,roc_curve,f1_score


In [32]:
print(accuracy_score(y_test,pred_arg)*100)
print(f1_score(y_test,pred_arg,average='macro')*100)
print(recall_score(y_test,pred_arg, average="weighted")*100)
print(precision_score(y_test,pred_arg, average="weighted")*100)

91.92077143601772
91.99723806014876
91.92077143601772
91.90354667789087
